Para comenzar, mostraremos el código de nuestro problema completo, basado en el usado por el profesor para el problema de localización y asignación

In [ ]:
#!/usr/bin/env python
#
# Problema de Localización y Asignación, abordado mediante relajación Lagrangeana

import gurobipy as gp
from gurobipy import GRB
import sys
import numpy as np
from time import time

NITER = 100

# Acá se definen los datos
#m = 50
#n = 1000
m = 50
n = 1000

M = range(m)
N = range(n)

np.random.seed(231199)

c = np.random.randint(3,7,size=(m,n))
f = np.random.randint(10,30,size=m)
h = np.random.uniform(10,16,size=(m,n))
H = np.random.uniform(800,1200,size=m)


# Se hacen las definiciones iniciales de los modelos

model = gp.Model('Location and Asignment with Lagrangian Relaxation')
model.setParam('OutputFlag', False) # turns off solver chatter

modelf = gp.Model('Location and Asignment with Lagrangian Relaxation')
modelf.setParam('OutputFlag', True) # turns on solver chatter


# Esta es la definición de variables y restricciones del problema completo original
xf = modelf.addVars(M,N,vtype=GRB.BINARY)
yf = modelf.addVars(M,vtype=GRB.BINARY)

#agregamos nuestra restricción 

rest_huella = modelf.addConstrs(((sum(xf[i,j]*h[i,j] for j in N) <= H[i]) for i in M), name = "HUELLA")
rest_demand = modelf.addConstrs(((sum(xf[i,j] for i in M) == 1) for j in N),name = "DEMANDF")
rest_assign = modelf.addConstrs(((sum(xf[i,j] for j in N) <= n*yf[i]) for i in M),name = "ASSIGN")

modelf.setObjective(sum(sum(c[i,j]*xf[i,j] for i in M) for j in N)+sum(f[i]*yf[i] for i in M),GRB.MINIMIZE)


modelf.update()
modelf.optimize()
valor_optimo = modelf.objVal



# Aquí se declara el problema relajado lineal y se resuelve
modelr = modelf.relax()
modelr.optimize()
# Acá se puede pedir modelr.getVars() y verificar que, efectivamente, las variables del problema
# relajado toman valores fraccionarios y no enteros.

valores = []

print( '\n Objectivo modelo relajado =', modelr.objVal,'\n')
print( '\n Objectivo modelo completo =', valor_optimo,'\n')

#for j in N:
    #print(j,(sum(x[i,j].X for i in M)))

Luego, mostramos el código que incluye la relajación lagrangeana del problema, en base al código que entregó el profesor:

In [ ]:
#!/usr/bin/env python
#
# Problema de Localización y Asignación, abordado mediante relajación Lagrangeana

import gurobipy as gp
from gurobipy import GRB
import sys
import numpy as np
from time import time

NITER = 100

# Acá se definen los datos
#m = 50
#n = 1000
m = 50
n = 1000

M = range(m)
N = range(n)

# Se utiliza como semilla el cumpleaños del integrante Vicente Besamat
np.random.seed(231199)

c = np.random.randint(3,7,size=(m,n))
f = np.random.randint(10,30,size=m)

# Acá se generan las matrices de datos de la huella de carbono, según una distribución Uniforme
h = np.random.uniform(10,16,size=(m,n))
H = np.random.uniform(800,1200,size=m)

# Se hacen las definiciones iniciales de los modelos

model = gp.Model('Location and Asignment with Lagrangian Relaxation')
model.setParam('OutputFlag', False) # turns off solver chatter

modelf = gp.Model('Location and Asignment with Lagrangian Relaxation')
modelf.setParam('OutputFlag', True) # turns on solver chatter


# Esta es la definición de variables y restricciones del problema completo original
xf = modelf.addVars(M,N,vtype=GRB.BINARY)
yf = modelf.addVars(M,vtype=GRB.BINARY)

# Se codifica la restricción de la huella de carbono agregada
rest_huella = modelf.addConstrs(((sum(xf[i,j]*h[i,j] for j in N) <= H[i]) for i in M), name = "HUELLA")
rest_demand = modelf.addConstrs(((sum(xf[i,j] for i in M) == 1) for j in N),name = "DEMANDF")
rest_assign = modelf.addConstrs(((sum(xf[i,j] for j in N) <= n*yf[i]) for i in M),name = "ASSIGN")

modelf.setObjective((sum(sum(c[i,j]*xf[i,j] for i in M) for j in N)+sum(f[i]*yf[i] for i in M)),GRB.MINIMIZE)


modelf.update()
modelf.optimize()
valor_optimo = modelf.objVal



# Aquí se declara el problema relajado lineal y se resuelve
modelr = modelf.relax()
modelr.optimize()
# Acá se puede pedir modelr.getVars() y verificar que, efectivamente, las variables del problema
# relajado toman valores fraccionarios y no enteros.



# Se rescatan las variables duales del problema relajado lineal
duals = [c.Pi for c in modelr.getConstrs()]

#for i in range(0,m):
#    for j in range(0,n):
#        print(i,j,xf[i,j].X)
        
#sys.exit()

#  Se definen las variables y restricciones de la función dual
x = model.addVars(m,n,vtype=GRB.BINARY)
y = model.addVars(M,vtype=GRB.BINARY)

rest_rel1 = model.addConstrs(((sum(x[i,j] for i in M) == 1) for j in N),name = "DEMANDF_REL")
rest_rel2 = model.addConstrs(((sum(x[i,j] for j in N) <= n*y[i]) for i in M),name = "ASSIGN_REL")
model.update()

# se calculan las variables duales iniciales, correspondientes a las restricciones de la huella de carbono
u = [0.0]*m
for i in M:
    u[i] = duals[i]

# Como cota se utiliza el valor óptimo del modelo completo, calculado previamente
u_cota = 3179
best = 0
# Este es el ciclo principal de la relajación lagrangeana

for k in range(1, NITER):
#   Se define la función objetivo de la función dual para esta iteración
    
    # model.setObjective((sum(sum(c[i,j]*x[i,j]) for i in M) for j in N) + (sum(f[i]*y[i]) for i in M) + (sum(u[i]*((sum(x[i,j]*h[i,j]) for j in N) - H[i])) for i in M),GRB.MINIMIZE)
    model.setObjective(sum(sum(c[i,j]*x[i,j] for i in M) for j in N) + sum(f[i]*y[i] for i in M)+ sum(u[i]*(sum(x[i,j]*h[i,j] for j in N) - H[i]) for i in M),GRB.MINIMIZE)

    model.update()
    model.optimize()

    dual_value = model.objVal
    print( 'iteration', k, 'obj =', dual_value)

    if dual_value > best: best = dual_value

#   Se calcula el subgradiente a usar

    sgrad = [0.0]*m
    for i in M:
        sgrad[i] = sum(x[i,j].X*h[i,j] for j in N) - H[i]
    
#   Acá se define el paso

#    s = 1.0/(k+1)**0.5
    rho = 1
    s = rho*abs(u_cota - dual_value)/(np.linalg.norm(sgrad)**2)

#   Se hace el avance en la dirección del subgradiente

    for i in M:
        u[i] = u[i] + s*(sgrad[i])

print( '\n Objectivo modelo relajado =', modelr.objVal,'\n')
print( '\n Objectivo modelo completo =', valor_optimo,'\n')
print( '\n Valor dual alcanzado =', best,'\n')

for j in N:
    print(j,(sum(x[i,j].X for i in M)))


A continuación, presentamos el código que incluye la resolución de la heurística, en base al que entregó el profesor como material del curso:

In [ ]:
#!/usr/bin/env python
#
# Problema de Localización y Asignación, abordado mediante relajación Lagrangeana

import gurobipy as gp
from gurobipy import GRB
import sys
import numpy as np
from time import time

NITER = 100

# Acá se definen los datos
m = 50
n = 1000
M = range(m)
N = range(n)

np.random.seed(231199)

#creamos h y H en base a lo descrito por el enunciado
c = np.random.randint(3,7,size=(m,n))
f = np.random.randint(10,30,size=m)
h = np.random.uniform(10,16,size=(m,n))
H = np.random.uniform(800,1200,size=m)

# Se hacen las definiciones iniciales de los modelos

model = gp.Model('Location and Asignment with Lagrangian Relaxation')
model.setParam('OutputFlag', False) # turns off solver chatter

modelf = gp.Model('Location and Asignment with Lagrangian Relaxation')
modelf.setParam('OutputFlag', True) # turns on solver chatter

modelh = gp.Model('Location and Asignment with Lagrangian Relaxation')
modelh.setParam('OutputFlag', False) # turns off solver chatter


# Esta es la definición de variables y restricciones del problema completo original
# Aquí definimos nuestra nueva restricción basada en las huellas (rest_huella)

xf = modelf.addVars(M,N,vtype=GRB.BINARY)
yf = modelf.addVars(M,vtype=GRB.BINARY)

rest_huella = modelf.addConstrs(((sum(xf[i,j]*h[i,j] for j in N) <= H[i]) for i in M), name = "HUELLA")
rest_demand = modelf.addConstrs(((sum(xf[i,j] for i in M) == 1) for j in N),name = "DEMANDF")
rest_assign = modelf.addConstrs(((sum(xf[i,j] for j in N) <= n*yf[i]) for i in M),name = "ASSIGN")


modelf.setObjective((sum(sum(c[i,j]*xf[i,j] for i in M) for j in N)+sum(f[i]*yf[i] for i in M)),GRB.MINIMIZE)

modelf.optimize()
valor_optimo = modelf.objVal



# Aquí se declara el problema relajado lineal y se resuelve
modelr = modelf.relax()
modelr.optimize()
# Acá se puede pedir modelr.getVars() y verificar que, efectivamente, las variables del problema
# relajado toman valores fraccionarios y no enteros.
# le agregamos nuestra restricción de huella

xh = modelh.addVars(M,N,vtype=GRB.BINARY)
rest_huellah = modelh.addConstrs(((sum(xh[i,j]*h[i,j] for j in N) <= H[i]) for i in M), name = "HUELLAH")
rest_demandh = modelh.addConstrs(((sum(xh[i,j] for i in M) == 1) for j in N),name = "DEMANDH")
rest_assignh = modelh.addConstrs(((sum(xh[i,j] for j in N) <= n) for i in M),name = "ASSIGNH")
modelh.setObjective((sum(sum(c[i,j]*xh[i,j] for i in M) for j in N)),GRB.MINIMIZE)


# Se rescatan las variables duales del problema relajado lineal
duals = [c.Pi for c in modelr.getConstrs()]

#for i in range(0,m):
#    for j in range(0,n):
#        print(i,j,xf[i,j].X)
        
#sys.exit()

#  Se definen las variables y restricciones  de la función dual
x = model.addVars(m,n,vtype=GRB.BINARY)
y = model.addVars(M,vtype=GRB.BINARY)

rest_rel1 = model.addConstrs(((sum(x[i,j] for i in M) == 1) for j in N),name = "DEMANDF_REL")
rest_rel2 = model.addConstrs(((sum(x[i,j] for j in N) <= n*y[i]) for i in M),name = "ASSIGN_REL")
model.update()

# se calculan las variables duales iniciales
u = [0.0]*m
for i in M:
    u[i] = duals[i]

#u_cota = valor_optimo
u_cota = 3179
best = 0
bestu = u_cota
# Este es el ciclo principal de la relajación lagrangeana
for k in range(1, NITER):
#   Se define la función objetivo de la función dual para esta iteración
    
    model.setObjective((sum(sum((c[i,j]+u[j])*x[i,j] for i in M) for j in N)+sum(f[i]*y[i] for i in M)),GRB.MINIMIZE)

    model.update()
    model.optimize()
    
#   Se calcula el valor de la función dual.

    dual_value = model.objVal

#    print( 'iteration', k, 'obj =', dual_value)
    if dual_value > best: best = dual_value

#   Se calcula el subgradiente a usar, el cual es el mismo que definimos en el documento latex

    sgrad = [0.0]*m
    for i in M:
        sgrad[i] = sum(x[i,j].X*h[i,j] for j in N) - H[i]
        
    cuenta = 0
    for i in M:
        rest_assignh[i].rhs = n*y[i].X
        cuenta = cuenta + y[i].X
    
    if cuenta == 0:
        rest_assignh[0].rhs = n
        
    modelh.update()
    modelh.optimize()
    u_cota = sum(sum(c[i,j]*xh[i,j].X for i in M) for j in N)+sum(f[i]*y[i].X for i in M)
    
    print( 'iteration', k, 'obj =', dual_value, 'cota = ',u_cota)
    
    if u_cota < bestu:
        bestu = u_cota
        
#   Acá se define el paso

#    s = 1.0/(k+1)**0.5
    rho = 1.0
    s = rho*abs(u_cota - dual_value)/(np.linalg.norm(sgrad)**2)

#   Se hace el avance en la dirección del subgradiente

    for i in M:
        u[i] = u[i] + s*(sgrad[i])

print( '\n Objectivo modelo relajado =', modelr.objVal,'\n')
print( '\n Objectivo modelo completo =', valor_optimo,'\n')
print( '\n Mejor valor dual alcanzado =', best,'\n')
print( '\n Mejor cota superior =', bestu,'\n')

#for j in N:
#    print(j,(sum(x[i,j].X for i in M)))


